# Notebook 2: Vertical Slice Gravity wave

In the first notebook, we looked at the shallow water equations. Gusto can also solve the incompressible Boussinesq equations and the compressible Euler equations. This notebook we will demonstrate running the compressible Euler equations in a 2D (x-z) domain, often called a 'vertical slice'. The variables in this system are the velocity $\textbf{u}$, the dry density $\rho$, and the (virtual dry) potential temperature $\theta$. The fluid is stratified and the background state is in hydrostatic balance. The initial conditions consist of a perturbation to $\theta$ in the centre of the domain. This setup is given in the 1994 paper by Skamarock and Klemp: https://journals.ametsoc.org/view/journals/mwre/122/11/1520-0493_1994_122_2623_eaaotk_2_0_co_2.xml.

As our standard first step, we begin by importing the required libraries and functions from Firedrake and Gusto:

In [1]:
from gusto import *
from firedrake import (as_vector, SpatialCoordinate, PeriodicIntervalMesh,
                       ExtrudedMesh, exp, sin, pi, Function)

Specify the time-step size, simulation duration, and frequency of output results.

In [2]:
dt = 6.
tmax = 3600.
dumpfreq = int(tmax / (2*dt))

For this vertical slice model, we first define a horizontal periodic interval mesh of length `L` with the number of cells equal to the number of columns we require in our final mesh. The periodic mesh is the extruded in the vertical direction by specifying the number of layers and their required height. We now have a two dimensional mesh with an aligned column structure.

In [3]:
nlayers = 10  # horizontal layers
columns = 150  # number of columns
L = 3.0e5
m = PeriodicIntervalMesh(columns, L)

H = 1.0e4  # Height position of the model top
mesh = ExtrudedMesh(m, layers=nlayers, layer_height=H/nlayers)

We now set up the `OutputParameters` class, specifying the path to the output directory `dirname`, passing in `dumpfreq` and specifying some additional output fields. The default behaviour is for the vtu output files to contain all of the prognostic fields but in this case we would like to visualise the evolution of the perturbation, so we specify `perturbation_fields=['theta', 'rho']`.

In [4]:
dirname = 'gravity_wave'
output = OutputParameters(dirname=dirname,
                          dumpfreq=dumpfreq,
                          perturbation_fields=['theta', 'rho'])

Now, define the parameters and diagnostics we want to record. We are interested in the temperature gradients, which is automatically computed using the `Gradient()` function. 

In [5]:
parameters = CompressibleParameters()
g = parameters.g
Tsurf = 300.

diagnostic_fields = [CourantNumber(), Gradient("u"),
                     Gradient("theta_perturbation"),
                     RichardsonNumber("theta", g/Tsurf), Gradient("theta")]

As always, we initialise our `state` that evolves with each time-step. The finite element family specified in the `CompressibleEulerEquations` class corresponds to the velocity space of the horizontal mesh. As this is a 1D mesh, Continuous Galerkin (CG) elements are used. As Gusto solves directly on the vertical slice domain, none of the fields are in the 2D CG space.

In [6]:
state = State(mesh,
              dt=dt,
              output=output,
              parameters=parameters,
              diagnostic_fields=diagnostic_fields)

eqns = CompressibleEulerEquations(state, "CG", 1)

Set up the initial conditions and define the thermodynamic constants.

In [7]:
# Initial conditions
u0 = state.fields("u")
rho0 = state.fields("rho")
theta0 = state.fields("theta")

# spaces
Vu = state.spaces("HDiv")
Vt = state.spaces("theta")
Vr = state.spaces("DG")

# Thermodynamic constants required for setting initial conditions
# and reference profiles
g = parameters.g
N = parameters.N
p_0 = parameters.p_0
c_p = parameters.cp
R_d = parameters.R_d
kappa = parameters.kappa

x, z = SpatialCoordinate(mesh)

# N^2 = (g/theta)dtheta/dz => dtheta/dz = theta N^2g => theta=theta_0exp(N^2gz)
Tsurf = 300.
thetab = Tsurf*exp(N**2*z/g)

theta_b = Function(Vt).interpolate(thetab)
rho_b = Function(Vr)

This next step takes the temperature profile and computes the corresponding density that will enforce a hydrostatic balance.

In [8]:
compressible_hydrostatic_balance(state, theta_b, rho_b)

We now set up the initial potential temperature profile. We apply a perturbation $\theta'$ to the background balanced $\theta_b$ defined above, where $\theta'$ is given by:
$$\theta' = \Delta \theta \frac{\sin\big(\frac{\pi z}{H}\big)}{1 + \frac{1}{a^2}(x-x_c)^2}$$

In [9]:
a = 5.0e3
deltaTheta = 1.0e-2
theta_pert = deltaTheta*sin(pi*z/H)/(1 + (x - L/2)**2/a**2)
theta0.interpolate(theta_b + theta_pert)
rho0.assign(rho_b)
u0.project(as_vector([20.0, 0.0]))

state.set_reference_profiles([('rho', rho_b),
                              ('theta', theta_b)])

A streamline upwind Petrov-Galerkin (SUPG) transport scheme will be applied to the potential temperature.

Set up the linear solver and the time-stepper. As per the first notebook, we use an Implicit Midpoint method for the velocity field, and an explicit SSPRK3 scheme for the other variables (density and potential temperature). This time, we need to specify the use of a compressible solver.

In [11]:
transported_fields = [ImplicitMidpoint(state, "u"),
                      SSPRK3(state, "rho"),
                      SSPRK3(state, "theta", options=SUPGOptions())]

# Set up linear solver
linear_solver = CompressibleSolver(state, eqns)

# build time stepper
stepper = SemiImplicitQuasiNewton(eqns, state, transported_fields, linear_solver=linear_solver)

Next, you would solve the equations! However, they take ages to run - do we run for a few time steps? The only problem is that it will create output which will overwrite the current results.

In [ ]:
# Here's what you would do...
# stepper.run(t=0, tmax=tmax)

To save you having to run this full simulation, results have already been generated and stored in the 'results/gravity_wave' subdirectory. We will show a visualisation of the potential temperature at the points we specified earlier. 

We set-up the visualisations that can be used in Jupyter notebook: